In [1]:
import numpy as np 
import pandas as pd

In [2]:
news = pd.read_csv('/notebooks/notebooks/data.csv', sep=',')
MAX_NEWS = 1000
DOCUMENT="Query"
TOPIC="Reasoning"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 500
#DOCUMENT="description"
#TOPIC="title"

#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

In [3]:
subset_news

,Query,Intermediate MQL,Reasoning
0,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
1,Query,Intermediate MQL,NaN
2,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
3,Query,Intermediate MQL,NaN
4,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,Step 1: Identify the components in the query\n...
5,Query,Intermediate MQL,NaN
6,sales in jan 2020 versus year ago,{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TI...,NaN


In [4]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [3]:
#!pip install langchain
#!pip install sentence_transformers
!pip install chromadb 
#!pip install -U pip setuptools
#!pip install --upgrade pip


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:0000:0100:01


In [6]:
df_loader = DataFrameLoader(subset_news, page_content_column=DOCUMENT)


In [7]:
df_document = df_loader.load()
display(df_document)

[Document(page_content='sales in jan 2020 versus year ago', metadata={'Intermediate MQL': "{'DATE VARIABLE': {'jan 2020': [{'CONVERTED TIME ELEMENT': 'january 2020', 'DATE RANGE': '2020/01/01 - 2020/01/31', 'ENTITY': 'Order Date'}]}, 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}", 'Reasoning': 'Step 1: Identify the components in the query\n- The query mentions "sales" which is a measure.\n- The query mentions "jan 2020" which is a date component.\n\nStep 2: Match the components to the context\n- "sales" can be matched to the "Sales" entity in the context under MEASURE.\n- "jan 2020" needs to be converted to a date range using the date reference provided.\n\nStep 3: Convert the date component\n- The query mentions "year ago" which needs to be calculated using the date reference.\n- The date reference has a start_date of \'01/01/2020\' and an end_date of \'15/09/2023\'.\n- Since the query asks for sales in Jan 2020, we don\'t need to consider the end_date for this calculation.\n- To calcu

In [8]:
from langchain.text_splitter import CharacterTextSplitter


In [9]:
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=10)
texts = text_splitter.split_documents(df_document)

In [10]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/pip_packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
chromadb_index = Chroma.from_documents(
    texts, embedding_function, persist_directory='./input'
)

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

In [ ]:
retriever = chromadb_index.as_retriever()


In [ ]:
model_id = "databricks/dolly-v2-3b" #my favourite textgeneration model for testing
task="text-generation"

In [ ]:
hf_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task=task,
    model_kwargs={
        "temperature": 0,
        "max_length": 256
    },
    pipeline_kwargs={
        "repetition_penalty":1.1
    }
)

In [ ]:
document_qa = RetrievalQA.from_chain_type(
    llm=hf_llm, chain_type="stuff", retriever=retriever
)

In [ ]:
response = document_qa.run("What is step 1 in Identify the components")

#Sample question for BBC Dataset. 
#response = document_qa.run("Who is going to meet boris johnson?")
display(response)